In [ ]:
from qiskit import QuantumRegister, QuantumCircuit

import numpy as np


In [ ]:
def initialize_state(oracle_domain: int):
    """accepts an integer and returns a circuit
    that initializes an input state of state_space
    qubits into an equal superpositional state
    """


    # we first determine the number of qubits necessary
    # describe our state in base 2
    # ie if the oracle accepts any integer in [0,7]
    # we want to use 3 qubits
    # if it accepts any integer [0,5] we also want
    # to use 3 qubits since this can not be represented
    # by 2 clasical bits

    qubits_req = int(np.ceil(np.log2(oracle_domain)))
    print(qubits_req)
    ret_circ = QuantumCircuit(qubits_req)

    for i in range(qubits_req):
        ret_circ.h(i)

    return ret_circ

def diffusion_operator()


In [19]:
def construct_groracle(true_value: int, max_value: int):
    if true_value >= max_value:
        raise Exception(
            f"{true_value} must be within [0, {max_value})"
        )
    
    return lambda x: x == true_value